In [6]:
import pandas as pd
!pip3 install fuzzywuzzy


path_spi = '/home/tdraths/DS_Projects/transfers_ver2/data/interim/spi_scores.csv'
path_transfers = '/home/tdraths/DS_Projects/transfers_ver2/data/interim/concatenated_datasets.csv'


spi = pd.read_csv(path_spi, index_col=0)
transfers = pd.read_csv(path_transfers, index_col=0)

In [2]:
transfers['league_name'].value_counts()

Serie A             5911
Championship        5512
Liga Nos            4837
Primera Division    3826
Premier League      3707
Ligue 1             3516
Eredivisie          3002
Premier Liga        2821
1 Bundesliga        2796
Name: league_name, dtype: int64

In [3]:
spi['league'].value_counts()

English League Championship    2228
Italy Serie A                  1900
Spanish Primera Division       1900
Barclays Premier League        1900
French Ligue 1                 1900
German Bundesliga              1530
Portuguese Liga                1224
Dutch Eredivisie               1224
Russian Premier Liga            955
Name: league, dtype: int64

In [4]:
league_names = {
    'Serie A': 'Italian Serie A',
    'Championship': 'English League Championship',
    'Liga Nos': 'Portuguese Liga',
    'Primera Division': 'Spanish La Liga',
    'Premier League': 'English Premier League',
    'Ligue 1': 'French Ligue 1',
    'Eredivisie': 'Dutch Eredivisie',
    'Premier Liga': 'Russian Premier Liga',
    '1 Bundesliga': 'German Bundesliga'
}

transfers['league_name'].replace(league_names, inplace=True)
transfers['league_name'].value_counts()

Italian Serie A                5911
English League Championship    5512
Portuguese Liga                4837
Spanish La Liga                3826
English Premier League         3707
French Ligue 1                 3516
Dutch Eredivisie               3002
Russian Premier Liga           2821
German Bundesliga              2796
Name: league_name, dtype: int64

In [5]:
spi['league'].value_counts()

league_names = {
    'Italy Serie A': 'Italian Serie A',
    'English League Championship': 'English League Championship',
    'Portuguese Liga': 'Portuguese Liga',
    'Spanish Primera Division': 'Spanish La Liga',
    'Barclays Premier League': 'English Premier League',
    'French Ligue 1': 'French Ligue 1',
    'Dutch Eredivisie': 'Dutch Eredivisie',
    'Russian Premier Liga': 'Russian Premier Liga',
    'German Bundesliga': 'German Bundesliga'
}

spi['league'].replace(league_names, inplace=True)
spi['league'].value_counts().sort_index(ascending=True)

Dutch Eredivisie               1224
English League Championship    2228
English Premier League         1900
French Ligue 1                 1900
German Bundesliga              1530
Italian Serie A                1900
Portuguese Liga                1224
Russian Premier Liga            955
Spanish La Liga                1900
Name: league, dtype: int64

### Using FuzzyWuzzy to clean team names

The FuzzyWuzzy library matches strings by similarity. I've never used it before, but I'm hoping it will help me match team names across the two datasets, and maybe even standardize the team names so that I can combine the two dataframes without creating a whole lot of new rows in the merged dataframe.

Let's go!

In [9]:
# I installed FuzzyWuzzy in the first cell in this notebook.
# Here, I'll import the library
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Ok, first import, first warning. Classic. I'll install `python-Levenshtein` below.

Briefly, ['the Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.'](https://en.wikipedia.org/wiki/Levenshtein_distance#Definition)

In [8]:
# Installing python-Levenshtein
!pip3 install python-Levenshtein

     |████████████████████████████████| 50 kB 1.4 MB/s eta 0:00:01
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=178101 sha256=54b909cf35169819258e9fdac4ed4d88937eee60a0700bb8abfb093eef282d9f
  Stored in directory: /home/tdraths/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


### String Matching with FuzzyWuzzy

FuzzyWuzzy lets us take two strings and match them, providing a similarity score. Here's an example:

In [11]:
Str_A = 'FuzzyWuzzy solves problems!'
Str_B = 'fuzzy wuzzy solves PROBLEMS.'

ratio = fuzz.ratio(Str_A.lower(), Str_B.lower())
print('Similarity score: {}'.format(ratio))

Similarity score: 95


### String replacement with FuzzyWuzzy

Aside from the `ratio()` function, FuzzyWuzzy also uses functions like `token_set_ratio()` and `token_sort_ratio()` that tokenize the strings and then manipulate them, e.g. changing letters to lowercase, before running `fuzz.ratio()`. For more information on FuzzyWuzzy, check out the [documentation](https://pypi.org/project/fuzzywuzzy/)


As I mentioned above, my two dataframes `transfers` and `spi` both have columns of team and league names. I used a manually created a dictionary of values to standardize the leagues. That was fine for nine values, but for team names, I have hundreds of unique strings to manage.

To use FuzzyWuzzy to 'find and replace' the team names, I'll cast the two columns of team names to lists. Then I'll create a function that uses FuzzyWuzzy to calculate a similarity score between two team names and returns a dictionary with the strings we need to use in replacement.

Let's go!

In [14]:
display(transfers.columns)
spi.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'transfer_movement', 'transfer_period', 'fee_cleaned', 'league_name',
       'year', 'season'],
      dtype='object')

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2'],
      dtype='object')

In [17]:
transfer_names = list(transfers.club_name.unique())
spi_names = list(spi.team1.unique())

def name_match(name, names_list, min_score=0):
    max_score = -1
    max_name = ''
    for a in names_list:
        score = fuzz.ratio(name, a)
        if (score > min_score) & (score > max_score):
            max_name = a
            max_score=score
    return (max_name, max_score)

names = []
for x in spi_names:
    match = name_match(x, transfer_names, 75)
    if match[1] >= 75:
        name = ('(' + str(x), str(match[0]) + ')')
        names.append(name)

name_dict = dict(names)
name_dict

{'(Bastia': 'SC Bastia)',
 '(AS Monaco': 'AS Monaco)',
 '(Hull City': 'Hull City)',
 '(Southampton': 'Southampton FC)',
 '(Burnley': 'Burnley FC)',
 '(Middlesbrough': 'Middlesbrough FC)',
 '(Crystal Palace': 'Crystal Palace)',
 '(Everton': 'Everton FC)',
 '(Manchester City': 'Manchester City)',
 '(Dijon FCO': 'Dijon FCO)',
 '(Montpellier': 'Montpellier HSC)',
 '(AFC Bournemouth': 'AFC Bournemouth)',
 '(AS Nancy Lorraine': 'AS Nancy-Lorraine)',
 '(Arsenal': 'Arsenal FC)',
 '(Chelsea': 'Chelsea FC)',
 '(Málaga': 'Málaga CF)',
 '(Manchester United': 'Manchester United)',
 '(Deportivo La Coruña': 'Deportivo de La Coruña)',
 '(Stoke City': 'Stoke City)',
 '(Watford': 'Watford FC)',
 '(West Bromwich Albion': 'West Bromwich Albion)',
 '(Swansea City': 'Swansea City)',
 '(Tottenham Hotspur': 'Tottenham Hotspur)',
 '(Nantes': 'FC Nantes)',
 '(AS Roma': 'AS Roma)',
 '(Barcelona': 'FC Barcelona)',
 '(Leicester City': 'Leicester City)',
 '(Stade Rennes': 'Stade Rennais FC)',
 '(Lorient': 'FC Lorie

#### OK! 
I have a dictionary of replacement names, which I'll use below to replace `team1` and `team2` names in `spi`.

In [18]:
transfers.club_name = transfers.club_name.replace(name_dict)
spi.team1 = spi.team1.replace(name_dict)
spi.team2 = spi.team2.replace(name_dict)